In [3]:
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import json
import random
import re
from collections import Counter

In [2]:
json_filenames = set(filename for filename in glob.glob("cleaned\*.json"))
json_temp_files = set(filename for filename in glob.glob("cleaned\*_temp.json"))

json_filenames -= json_temp_files

In [3]:
def extract_last_digits_from_url(url):
  """
  Извлекает последние цифры из URL-адреса, используя регулярное выражение.

  Аргументы:
    url: URL-адрес в виде строки.

  Возвращает:
    Строка, содержащая последние цифры из URL-адреса, или None, если цифры не найдены.
  """
  match = re.search(r'/(\d+)/?$', url)
  if match:
    return int(match.group(1))
  else:
    return None

In [4]:
data = {}

for filename in json_filenames:
    with open(filename, "r", encoding="utf-8") as file:
        category_data = json.load(file)

    if len(category_data["products"]) > 15:
        data[category_data["category_name"]] = category_data["products"]

In [ ]:
dict_products = {}
arr_exists = []

for filename in json_filenames:
    with open(filename, "r", encoding="utf-8") as file:
        category_data = json.load(file)
    
    # print(category_data["category_name"])

    for product in category_data["products"]:
        product["category"] = [category_data["category_name"]]
        # print(product["link"])
        digits = extract_last_digits_from_url(product["link"])

        if digits not in dict_products:
            dict_products[digits] = product
        else:
            dict_products[digits]["category"] = list(set(dict_products[digits]["category"] + [category_data["category_name"]]))
            # if len(dict_products[digits]["category"]) > 1:
            #     print(dict_products[digits]["title"], dict_products[digits]["category"])
            # compare_products(dict_products[digits], product)       

In [ ]:
s = set()

while len(s)!= 100:
    s = set()
    random_choices = {}
    for category in random.choices(list(data.keys()), k = 20):
        if category not in random_choices:
            random_choices[category] = []

        for product in random.choices(data[category], k = 5 ):
            random_choices[category].append(extract_last_digits_from_url(product["link"]))

    for category in random_choices:
        s.update(random_choices[category])
    
len(s)

res = [dict_products[id] for id in s]
len(res)

products = res

In [ ]:
# Убираем размерности в конец строки
for product in products:
    old_attributes = []
    new_attributes = {}

    for attribute, value in product["attributes"].items():
        if ", " in attribute:
            arr = attribute.split(', ')

            new_attribute = arr[0]
            dims = arr[1:]

            value = f'{value} {", ".join(dims)}'
            
            new_attributes[new_attribute] = value
            old_attributes.append(attribute)

    
    product["attributes"].update(new_attributes)

    for old_attribute in old_attributes:
        product['attributes'].pop(old_attribute, None)

with open("choice.json", "w", encoding="utf-8") as file:
    json.dump(products, file, ensure_ascii=False)

Counter({'Школьные тетради (12-24 листов)': 5,
         'Школьные блокноты и тетради': 5,
         'Запасные части (ЗИП) для принтеров': 5,
         'Кулеры для процессора': 5,
         'Элементы бенч-систем': 5,
         'Папки-планшеты': 5,
         'Папки-конверты': 5,
         'Ручные уровни и угольники': 5,
         'Пуфы': 5,
         'Ручки гелевые': 5,
         'Ручки': 5,
         'Настольные подставки и наборы': 5,
         'Ламинаторы': 5,
         'Офисные защитные экраны на струбцинах': 5,
         'Рабочая непромокаемая спецодежда': 5,
         'Шкафы для одежды': 5,
         'Наборы посуды для приготовления': 5,
         'Кастрюли, сотейники, котлы, мантоварки': 5,
         'Цветные карандаши, фломастеры': 5,
         'Карандаши цветные': 5,
         'Профессиональные акустические системы': 5,
         '3D принтеры, сканеры и расходные материалы': 5,
         'Магнитно-маркерные доски': 5,
         'Стеновые панели для кухни': 5,
         'Тетради и блокноты': 4,
       

In [7]:
# Загрузка готового датасета
with open("choice.json", "r", encoding="utf-8") as file:
    products = json.load(file)

In [8]:
c = Counter()

for product in products:
    c.update(product["category"])
c

Counter({'Школьные тетради (12-24 листов)': 5,
         'Школьные блокноты и тетради': 5,
         'Запасные части (ЗИП) для принтеров': 5,
         'Кулеры для процессора': 5,
         'Элементы бенч-систем': 5,
         'Папки-планшеты': 5,
         'Папки-конверты': 5,
         'Ручные уровни и угольники': 5,
         'Пуфы': 5,
         'Ручки гелевые': 5,
         'Ручки': 5,
         'Настольные подставки и наборы': 5,
         'Ламинаторы': 5,
         'Офисные защитные экраны на струбцинах': 5,
         'Рабочая непромокаемая спецодежда': 5,
         'Шкафы для одежды': 5,
         'Наборы посуды для приготовления': 5,
         'Кастрюли, сотейники, котлы, мантоварки': 5,
         'Цветные карандаши, фломастеры': 5,
         'Карандаши цветные': 5,
         'Профессиональные акустические системы': 5,
         '3D принтеры, сканеры и расходные материалы': 5,
         'Магнитно-маркерные доски': 5,
         'Стеновые панели для кухни': 5,
         'Тетради и блокноты': 4,
       

In [6]:
keys = [item for item in c.keys()]
values = [item for item in c.values()]

# Создание DataFrame (необязательно, но удобно для plotly)
import pandas as pd
df = pd.DataFrame({'Атрибут': keys, 'Количество': values})

# Создание графика
fig = px.bar(
    df,
    x='Атрибут',
    y='Количество',
    title=f'График распределения категорий',
    color='Количество',  # Цветовая шкала
    text='Количество'    # Отображение значений над столбцами
)

# Настройка визуализации
fig.update_traces(texttemplate='%{text:.2s}', textposition='auto')
fig.update_layout(showlegend=False)

# Отображение графика
fig.show()